실시간 위험지수 산출

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from bs4 import BeautifulSoup
import requests
from urllib import parse

print(tf.__version__)

2.5.0


In [2]:
import datetime
from datetime import timedelta

now = datetime.datetime.now()
day = now.strftime('%Y%m%d')
mon_num = float(now.strftime('%m'))
day_num = float(now.strftime('%d'))

delta = timedelta(hours=1)
hour_now = now - delta
hour = hour_now.strftime('%H')+'00'

print(now)

2021-10-01 15:22:00.682924


In [3]:
def crawling_weather(date, time, nx, ny):
    param = {
        'serviceKey' : 'dLeJY56I9i5QWd6UksGvqxOsIKaVQ3Aa9rNJZW3rOlPd6FBEoTlhtmmjMsghEicTUqb64WaPkNqv852BGEBLIA=='
        , 'numOfRows' : '14'
        , 'pageNo' : '1'
        , 'dataType' : 'XML'
        , 'base_date' : date
        , 'base_time' : time
        , 'nx' : nx
        , 'ny' : ny
    }
    p_param = parse.urlencode(param)

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'

    request_info = requests.get(url + '?' + p_param)
    soup = BeautifulSoup(request_info.text, 'html.parser')
    
    return soup

In [4]:
def listing_bs_weather(raw_bs, city):
    items = raw_bs.select('item')
    return_arr = [city]
            
    for item in items:
        temp_arr = []
        sel_cat = item.select_one('category').string
        sel_data = item.select_one('obsrvalue').string
        if sel_cat == 'T1H':
            temp_arr.append(sel_data)
        elif sel_cat == 'RN1':
            if sel_data == '0':
                sel_data = -1
            temp_arr.append(sel_data)
        elif sel_cat == 'REH':
            temp_arr.append(sel_data)
        elif sel_cat == 'WSD':
            temp_arr.append(sel_data)    
        return_arr.extend(temp_arr)
        
    return return_arr

In [5]:
arr_col = [ 'city', 'humid', 'rain', 'avg temp', 'wind']

loc_arr = {'서울' : [60,127]
           , '부산' : [98,76]
           , '대구' : [89,90]
           , '인천' : [55,124]
           , '광주' : [58,74]
           , '대전' : [67,100]
           , '울산' : [102,84]
           , '경기' : [60,120]
           , '강원' : [73,134]
           , '충북' : [69,107]
           , '충남' : [68,100]
           , '전북' : [63,89]
           , '전남' : [51,67]
           , '경북' : [89,91]
           , '경남' : [91,77]
           , '제주' : [52,38]}

weather_list = []

for key, value in loc_arr.items():
    weather_html = crawling_weather(day, hour, value[0], value[1])
    weather_list.append(listing_bs_weather(weather_html, key))
    print('loc :', key ,'complete')
    
weather_pd = pd.DataFrame(weather_list)
weather_pd.columns = arr_col

weather_pd.head()

loc : 서울 complete
loc : 부산 complete
loc : 대구 complete
loc : 인천 complete
loc : 광주 complete
loc : 대전 complete
loc : 울산 complete
loc : 경기 complete
loc : 강원 complete
loc : 충북 complete
loc : 충남 complete
loc : 전북 complete
loc : 전남 complete
loc : 경북 complete
loc : 경남 complete
loc : 제주 complete


,city,humid,rain,avg temp,wind
0,서울,43,-1,27,3.1
1,부산,59,-1,26.7,0.6
2,대구,52,-1,27.5,1.3
3,인천,57,-1,27,1.2
4,광주,53,-1,28.2,0.9


In [6]:
def crawling_dust(loc_name):
    param = {
        'serviceKey' : 'dLeJY56I9i5QWd6UksGvqxOsIKaVQ3Aa9rNJZW3rOlPd6FBEoTlhtmmjMsghEicTUqb64WaPkNqv852BGEBLIA=='
        , 'returnType' : 'XML'
        , 'numOfRows' : '100'
        , 'pageNo' : '1'
        , 'sidoName' : loc_name
        , 'ver' : '1.0'
    }
    p_param = parse.urlencode(param)

    url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'

    request_info = requests.get(url + '?' + p_param)
    soup = BeautifulSoup(request_info.text, 'html.parser')
    
    return soup

In [7]:
def listing_bs_dust(raw_bs):
    items = raw_bs.select('item')
    return_arr = []
    
    arr_code = [
        'so2value' , 'o3value', 'no2value' , 'pm10value' , 'pm25value'
    ]
    
    for item in items:
        temp_arr = []
        for key in arr_code:
            sel_data = item.select_one(key).string
            if sel_data is not None:
                temp_arr.append(sel_data)
            else:
                temp_arr.append('-')
        return_arr.append(temp_arr)
        
    return return_arr

In [8]:
arr_col_dust = [ 'city', 'SO2', 'O3', 'NO2', 'PM10', 'PM25' ]

loc_names = ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']

dust_list = []

for loc_name in loc_names:
    city_list = [loc_name]
    
    dust_html = crawling_dust(loc_name)
    html_list = listing_bs_dust(dust_html)
    temp_list = [0,0,0,0,0]
    temp_num = [0,0,0,0,0]
    for value_list in html_list:
        for number in range(5):
            if value_list[number] != '-':
                temp_list[number] = (temp_list[number]*temp_num[number] + float(value_list[number])) / (temp_num[number]+1)
                temp_num[number] = temp_num[number]+1
    city_list.extend(temp_list)
    dust_list.append(city_list)
    print('loc :', loc_name ,'complete')
    
dust_pd = pd.DataFrame(dust_list)
dust_pd.columns = arr_col_dust

dust_pd.head()

loc : 서울 complete
loc : 부산 complete
loc : 대구 complete
loc : 인천 complete
loc : 광주 complete
loc : 대전 complete
loc : 울산 complete
loc : 경기 complete
loc : 강원 complete
loc : 충북 complete
loc : 충남 complete
loc : 전북 complete
loc : 전남 complete
loc : 경북 complete
loc : 경남 complete
loc : 제주 complete


,city,SO2,O3,NO2,PM10,PM25
0,서울,0.003263,0.053769,0.026385,44.461538,22.179487
1,부산,0.003357,0.060719,0.014750,21.483871,10.866667
2,대구,0.001579,0.062947,0.010474,22.200000,11.450000
3,인천,0.003176,0.070029,0.021971,50.764706,27.000000
4,광주,0.003308,0.078923,0.013231,34.461538,24.333333


In [9]:
def crawling_weather_forcast(date, time, nx, ny):
    param = {
        'serviceKey' : 'dLeJY56I9i5QWd6UksGvqxOsIKaVQ3Aa9rNJZW3rOlPd6FBEoTlhtmmjMsghEicTUqb64WaPkNqv852BGEBLIA=='
        , 'numOfRows' : '200'
        , 'pageNo' : '1'
        , 'dataType' : 'XML'
        , 'base_date' : date
        , 'base_time' : '0200'
        , 'nx' : nx
        , 'ny' : ny
    }
    p_param = parse.urlencode(param)

    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'

    request_info = requests.get(url + '?' + p_param)
    soup = BeautifulSoup(request_info.text, 'html.parser')
    
    return soup

In [10]:
def listing_bs_weather_forcast(raw_bs, city):
    items = raw_bs.select('item')
    return_arr = [city]
            
    for item in items:
        temp_arr = []
        sel_cat = item.select_one('category').string
        sel_data = item.select_one('fcstValue').string
        if sel_cat == 'TMX':
            temp_arr.append(sel_data)
        elif sel_cat == 'TMN':
            temp_arr.append(sel_data)
        return_arr.extend(temp_arr)
        
    return return_arr

In [11]:
arr_col = [ 'city', 'min temp', 'max temp']

loc_arr = {'서울' : [60,127]
           , '부산' : [98,76]
           , '대구' : [89,90]
           , '인천' : [55,124]
           , '광주' : [58,74]
           , '대전' : [67,100]
           , '울산' : [102,84]
           , '경기' : [60,120]
           , '강원' : [73,134]
           , '충북' : [69,107]
           , '충남' : [68,100]
           , '전북' : [63,89]
           , '전남' : [51,67]
           , '경북' : [89,91]
           , '경남' : [91,77]
           , '제주' : [52,38]}

weather_forcast_list = []

for key, value in loc_arr.items():
    weather_forcast_html = crawling_weather_forcast(day, hour, value[0], value[1])
    weather_forcast_list.append(listing_bs_weather_forcast(weather_forcast_html, key))
    print('loc :', key ,'complete')
    
weather_forcast_pd = pd.DataFrame(weather_forcast_list)
weather_forcast_pd.columns = arr_col

weather_forcast_pd.head()

loc : 서울 complete
loc : 부산 complete
loc : 대구 complete
loc : 인천 complete
loc : 광주 complete
loc : 대전 complete
loc : 울산 complete
loc : 경기 complete
loc : 강원 complete
loc : 충북 complete
loc : 충남 complete
loc : 전북 complete
loc : 전남 complete
loc : 경북 complete
loc : 경남 complete
loc : 제주 complete


,city,min temp,max temp
0,서울,18.0,26.0
1,부산,17.0,29.0
2,대구,16.0,29.0
3,인천,18.0,25.0
4,광주,17.0,28.0


과거 데이터 로드 및 모델 사용

In [12]:
# weather_pd, dust_pd, weather_forcast_pd

temp_pd = pd.merge(weather_pd, weather_forcast_pd,on='city')
today_pd = pd.merge(temp_pd, dust_pd,on='city')
today_pd = today_pd.astype({'max temp':float, 'min temp':float, 'humid':float, 'rain':float, 'avg temp':float, 'wind':float, 'SO2':float, 'O3':float, 'NO2':float, 'PM10':float, 'PM25':float})
today_pd['diff temp'] = today_pd['max temp'] - today_pd['min temp']
today_pd.drop(['max temp', 'min temp'], axis = 1, inplace = True)
today_pd['mon'] = mon_num
today_pd['day'] = day_num

city = today_pd['city']
today_pd['city1'] = (city == '서울')*1.0
today_pd['city2'] = (city == '부산')*1.0
today_pd['city3'] = (city == '대구')*1.0
today_pd['city4'] = (city == '인천')*1.0
today_pd['city5'] = (city == '광주')*1.0
today_pd['city6'] = (city == '대전')*1.0
today_pd['city7'] = (city == '울산')*1.0
today_pd['city8'] = (city == '경기')*1.0
today_pd['city9'] = (city == '강원')*1.0
today_pd['city10'] = (city == '충북')*1.0
today_pd['city11'] = (city == '충남')*1.0
today_pd['city12'] = (city == '전북')*1.0
today_pd['city13'] = (city == '전남')*1.0
today_pd['city14'] = (city == '경북')*1.0
today_pd['city15'] = (city == '경남')*1.0
today_pd['city16'] = (city == '제주')*1.0

today_pd.head()

,city,humid,rain,avg temp,wind,SO2,O3,NO2,PM10,PM25,...,city7,city8,city9,city10,city11,city12,city13,city14,city15,city16
0,서울,43.0,-1.0,27.0,3.1,0.003263,0.053769,0.026385,44.461538,22.179487,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,부산,59.0,-1.0,26.7,0.6,0.003357,0.060719,0.014750,21.483871,10.866667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,대구,52.0,-1.0,27.5,1.3,0.001579,0.062947,0.010474,22.200000,11.450000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,인천,57.0,-1.0,27.0,1.2,0.003176,0.070029,0.021971,50.764706,27.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,광주,53.0,-1.0,28.2,0.9,0.003308,0.078923,0.013231,34.461538,24.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
CSV_ROOT = './savedata/avg_data_save.csv'

pre_save_data = pd.read_csv(CSV_ROOT)
pre_save_data = pre_save_data[['city', 'date','SO2', 'O3', 'NO2', 'PM10', 'PM25', 'avg temp', 'max temp', 'min temp', 'rain', 'wind', 'humid']]
pre_save_data.sort_values(by=['date','city'])
pre_save_data['diff temp'] = pre_save_data['max temp'] - pre_save_data['min temp']
pre_save_data.drop(['max temp', 'min temp'], axis = 1, inplace = True)
pre_save_data.fillna({'rain':-1}, inplace = True)

pre_save_data.head(n=5)

,city,date,SO2,O3,NO2,PM10,PM25,avg temp,rain,wind,humid,diff temp
0,강원,2021-09-30,0.001667,0.020667,0.004500,9.666667,4.666667,19.533333,-1.00,1.200000,83.966667,10.833333
1,경기,2021-09-30,0.002000,0.016000,0.023750,17.500000,7.000000,19.000000,-1.00,0.833333,89.766667,10.666667
2,경남,2021-09-30,0.002467,0.028667,0.011900,11.000000,7.666667,21.250000,16.95,1.000000,85.750000,11.400000
3,경북,2021-09-30,0.002700,0.030667,0.006233,15.333333,6.666667,20.300000,1.80,0.900000,83.300000,11.400000
4,광주,2021-09-30,0.003067,0.036667,0.010667,14.666667,8.333333,23.500000,2.00,NaN,67.900000,15.200000


In [14]:
import datetime
from datetime import timedelta

now = datetime.datetime.now()

delta = timedelta(days=1)
day_one = now - delta
day_one_str = day_one.strftime('%Y%m%d')
day_one_dash = day_one.strftime('%Y-%m-%d')
delta = timedelta(days=2)
day_two = now - delta
day_two_str = day_two.strftime('%Y%m%d')
day_two_dash = day_two.strftime('%Y-%m-%d')
delta = timedelta(days=3)
day_three = now - delta
day_three_str = day_three.strftime('%Y%m%d')
day_three_dash = day_three.strftime('%Y-%m-%d')

day_list = [day_one_dash, day_two_dash, day_three_dash]

In [15]:
def make_day_df(all_df, date, col_name):
    find_df = all_df['date'] == date
    return_df = all_df[find_df].copy()
    return_df.rename(columns = {'SO2' : col_name[0], 'O3':col_name[1], 'NO2':col_name[2], 'PM10':col_name[3], 'PM25':col_name[4]
                                , 'avg temp':col_name[5], 'rain':col_name[6], 'wind':col_name[7], 'humid':col_name[8], 'diff temp':col_name[9]}
                     , inplace = True)
    
    return return_df

In [16]:
pre_date = [day_one_dash, day_two_dash, day_three_dash]
pre_col_name = [['SO2 -1','O3 -1','NO2 -1','PM10 -1','PM25 -1', 'avg temp -1','rain -1','wind -1','humid -1','diff temp -1']
                ,['SO2 -2','O3 -2','NO2 -2','PM10 -2','PM25 -2', 'avg temp -2','rain -2','wind -2','humid -2','diff temp -2']
                ,['SO2 -3','O3 -3','NO2 -3','PM10 -3','PM25 -3', 'avg temp -3','rain -3','wind -3','humid -3','diff temp -3']]

pre_pd_list = []

for i in range(3):
    temp_pd = make_day_df(pre_save_data, pre_date[i], pre_col_name[i])
    pre_pd_list.append(temp_pd)
    
merge1 = pd.merge(today_pd, pre_pd_list[0],on='city')
merge2 = pd.merge(pre_pd_list[1], pre_pd_list[2],on='city')
all_pd = pd.merge(merge1, merge2,on='city')
    
all_pd.head(n=10)

,city,humid,rain,avg temp,wind,SO2,O3,NO2,PM10,PM25,...,SO2 -3,O3 -3,NO2 -3,PM10 -3,PM25 -3,avg temp -3,rain -3,wind -3,humid -3,diff temp -3
0,서울,43.0,-1.0,27.0,3.1,0.003263,0.053769,0.026385,44.461538,22.179487,...,0.002300,0.018333,0.015000,13.000000,4.666667,21.500000,1.300000,2.200000,75.100000,4.800000
1,부산,59.0,-1.0,26.7,0.6,0.003357,0.060719,0.014750,21.483871,10.866667,...,0.003733,0.032000,0.013167,16.333333,7.000000,22.400000,0.000000,2.600000,77.300000,6.300000
2,대구,52.0,-1.0,27.5,1.3,0.001579,0.062947,0.010474,22.200000,11.450000,...,0.001067,0.030000,0.009267,19.666667,10.333333,21.800000,-1.000000,2.000000,75.800000,7.000000
3,인천,57.0,-1.0,27.0,1.2,0.003176,0.070029,0.021971,50.764706,27.000000,...,0.002667,0.014000,0.023800,11.000000,6.000000,21.300000,3.500000,2.200000,71.000000,3.600000
4,광주,53.0,-1.0,28.2,0.9,0.003308,0.078923,0.013231,34.461538,24.333333,...,0.003467,0.032000,0.011233,18.333333,10.666667,22.700000,0.300000,1.100000,83.800000,5.000000
5,대전,56.0,-1.0,27.8,1.0,0.002500,0.005308,0.015077,24.230769,10.583333,...,0.002400,0.030000,0.011733,12.000000,6.666667,22.000000,0.700000,1.600000,82.900000,6.000000
6,울산,69.0,-1.0,24.6,3.0,0.005143,0.053190,0.014857,19.428571,8.894737,...,0.002067,0.030000,0.008100,14.333333,6.333333,21.400000,0.000000,1.400000,84.800000,6.700000
7,경기,48.0,-1.0,27.6,2.8,0.003038,0.063101,0.017033,45.677778,21.086022,...,0.001950,0.019500,0.014450,13.500000,5.500000,20.066667,1.933333,1.566667,85.900000,6.400000
8,강원,52.0,-1.0,26.7,1.0,0.002520,0.046303,0.004758,25.030303,11.363636,...,0.001667,0.020333,0.003867,8.666667,2.666667,20.166667,0.500000,1.000000,81.233333,6.600000
9,충북,66.0,-1.0,24.8,0.7,0.002227,0.061484,0.012258,32.000000,17.285714,...,0.001867,0.026667,0.006600,11.333333,6.000000,20.600000,1.466667,1.166667,79.666667,6.933333


In [17]:
city_list = all_pd.pop('city')
all_pd.drop(['date','date_y','date_x'], axis = 1, inplace = True)
all_pd.astype(float)
print(city_list)

0     서울
1     부산
2     대구
3     인천
4     광주
5     대전
6     울산
7     경기
8     강원
9     충북
10    충남
11    전북
12    전남
13    경북
14    경남
15    제주
Name: city, dtype: object


In [18]:
# (x - x.mean()) / x.std()

all_pd['mon'] = (all_pd['mon'] - 6.575982) / 3.433015
all_pd['day'] = (all_pd['day'] - 16.104773) / 8.690128
all_pd['SO2'] = (all_pd['SO2'] - 0.004082) / 0.001809
all_pd['O3'] = (all_pd['O3'] - 0.028704) / 0.012174
all_pd['NO2'] = (all_pd['NO2'] - 0.019958) / 0.010027
all_pd['PM10'] = (all_pd['PM10'] - 43.616678) / 22.753832
all_pd['PM25'] = (all_pd['PM25'] - 24.897566) / 13.517440
all_pd['avg temp'] = (all_pd['avg temp'] - 13.882459) / 9.773169
all_pd['rain'] = (all_pd['rain'] - 3.817842) / 12.025526
all_pd['wind'] = (all_pd['wind'] - 2.032761) / 1.028838
all_pd['humid'] = (all_pd['humid'] - 67.699973) / 16.076195
all_pd['diff temp'] = (all_pd['diff temp'] - 9.548801) / 3.727797
all_pd['SO2 -1'] = (all_pd['SO2 -1'] - 0.004065) / 0.001809
all_pd['SO2 -2'] = (all_pd['SO2 -2'] - 0.004069) / 0.001809
all_pd['SO2 -3'] = (all_pd['SO2 -3'] - 0.004069) / 0.001809
all_pd['O3 -1'] = (all_pd['O3 -1'] - 0.028892) / 0.012289
all_pd['O3 -2'] = (all_pd['O3 -2'] - 0.028892) / 0.012289
all_pd['O3 -3'] = (all_pd['O3 -3'] - 0.028892) / 0.012289
all_pd['NO2 -1'] = (all_pd['NO2 -1'] - 0.019474) / 0.009897
all_pd['NO2 -2'] = (all_pd['NO2 -2'] - 0.019474) / 0.009897
all_pd['NO2 -3'] = (all_pd['NO2 -3'] - 0.019474) / 0.009897
all_pd['PM10 -1'] = (all_pd['PM10 -1'] - 43.449786) / 22.739767
all_pd['PM10 -2'] = (all_pd['PM10 -2'] - 43.449786) / 22.739767
all_pd['PM10 -3'] = (all_pd['PM10 -3'] - 43.449786) / 22.739767
all_pd['PM25 -1'] = (all_pd['PM25 -1'] - 24.813919) / 13.445748
all_pd['PM25 -2'] = (all_pd['PM25 -2'] - 24.813919) / 13.445748
all_pd['PM25 -3'] = (all_pd['PM25 -3'] - 24.813919) / 13.445748
all_pd['avg temp -1'] = (all_pd['avg temp -1'] - 13.903277) / 9.743491
all_pd['avg temp -2'] = (all_pd['avg temp -2'] - 13.903277) / 9.743491
all_pd['avg temp -3'] = (all_pd['avg temp -3'] - 13.903277) / 9.743491
all_pd['rain -1'] = (all_pd['rain -1'] - 3.758432) / 11.836105
all_pd['rain -2'] = (all_pd['rain -2'] - 3.758432) / 11.836105
all_pd['rain -3'] = (all_pd['rain -3'] - 3.758432) / 11.836105
all_pd['wind -1'] = (all_pd['wind -1'] - 2.029850) / 1.027535
all_pd['wind -2'] = (all_pd['wind -2'] - 2.029850) / 1.027535
all_pd['wind -3'] = (all_pd['wind -3'] - 2.029850) / 1.027535
all_pd['humid -1'] = (all_pd['humid -1'] - 67.903288) / 15.974801
all_pd['humid -2'] = (all_pd['humid -2'] - 67.903288) / 15.974801
all_pd['humid -3'] = (all_pd['humid -3'] - 67.903288) / 15.974801
all_pd['diff temp -1'] = (all_pd['diff temp -1'] - 9.541868) / 3.724790
all_pd['diff temp -2'] = (all_pd['diff temp -2'] - 9.541868) / 3.724790
all_pd['diff temp -3'] = (all_pd['diff temp -3'] - 9.541868) / 3.724790
all_pd['city1'] = (all_pd['city1'] - 0.062923) / 0.242832
all_pd['city2'] = (all_pd['city2'] - 0.064407) / 0.245484
all_pd['city3'] = (all_pd['city3'] - 0.063338) / 0.243578
all_pd['city4'] = (all_pd['city4'] - 0.063457) / 0.243791
all_pd['city5'] = (all_pd['city5'] - 0.062982) / 0.242938
all_pd['city6'] = (all_pd['city6'] - 0.063635) / 0.244109
all_pd['city7'] = (all_pd['city7'] - 0.065001) / 0.246534
all_pd['city8'] = (all_pd['city8'] - 0.063695) / 0.244215
all_pd['city9'] = (all_pd['city9'] - 0.062982) / 0.242938
all_pd['city10'] = (all_pd['city10'] - 0.063398) / 0.243684
all_pd['city11'] = (all_pd['city11'] - 0.056927) / 0.231711
all_pd['city12'] = (all_pd['city12'] - 0.059124) / 0.235863
all_pd['city13'] = (all_pd['city13'] - 0.064526) / 0.245694
all_pd['city14'] = (all_pd['city14'] - 0.056868) / 0.231597
all_pd['city15'] = (all_pd['city15'] - 0.063101) / 0.243152
all_pd['city16'] = (all_pd['city16'] - 0.063635) / 0.244109

all_pd = all_pd.fillna(0)

all_pd = all_pd[['mon', 'day', 'SO2', 'O3', 'NO2', 'PM10', 'PM25', 'avg temp', 'rain', 'wind', 'humid', 'diff temp', 'SO2 -1'
                 , 'SO2 -2', 'SO2 -3', 'O3 -1', 'O3 -2', 'O3 -3', 'NO2 -1', 'NO2 -2', 'NO2 -3', 'PM10 -1', 'PM10 -2', 'PM10 -3'
                 , 'PM25 -1', 'PM25 -2', 'PM25 -3', 'avg temp -1', 'avg temp -2', 'avg temp -3', 'rain -1', 'rain -2', 'rain -3'
                 , 'wind -1', 'wind -2', 'wind -3', 'humid -1', 'humid -2', 'humid -3', 'diff temp -1', 'diff temp -2'
                 , 'diff temp -3', 'city1', 'city2', 'city3', 'city4', 'city5', 'city6', 'city7', 'city8', 'city9', 'city10'
                 , 'city11', 'city12', 'city13', 'city14', 'city15', 'city16']]

all_pd.head(n=10)

,mon,day,SO2,O3,NO2,PM10,PM25,avg temp,rain,wind,...,city7,city8,city9,city10,city11,city12,city13,city14,city15,city16
0,0.997379,-1.738153,-0.452649,2.058915,0.640931,0.037130,-0.201079,1.342199,-0.400635,1.037325,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
1,0.997379,-1.738153,-0.400695,2.629764,-0.519398,-0.972707,-1.037985,1.311503,-0.400635,-1.392601,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
2,0.997379,-1.738153,-1.383666,2.812828,-0.945878,-0.941234,-0.994831,1.393360,-0.400635,-0.712222,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
3,0.997379,-1.738153,-0.500569,3.394563,0.200717,0.314146,0.155535,1.342199,-0.400635,-0.809419,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
4,0.997379,-1.738153,-0.428031,4.125109,-0.670912,-0.402356,-0.041741,1.464984,-0.400635,-1.101010,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
5,0.997379,-1.738153,-0.874516,-1.921826,-0.486793,-0.851984,-1.058946,1.424056,-0.400635,-1.003813,...,-0.263659,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
6,0.997379,-1.738153,0.586433,2.011375,-0.508712,-1.063034,-1.183865,1.096629,-0.400635,0.940128,...,3.792576,-0.260815,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
7,0.997379,-1.738153,-0.577391,2.825458,-0.291716,0.090583,-0.281972,1.403592,-0.400635,0.745734,...,-0.263659,3.833937,-0.259251,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
8,0.997379,-1.738153,-0.863460,1.445624,-1.515949,-0.816846,-1.001220,1.311503,-0.400635,-1.003813,...,-0.263659,-0.260815,3.857025,-0.260165,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683
9,0.997379,-1.738153,-1.025278,2.692613,-0.767920,-0.510537,-0.563113,1.117093,-0.400635,-1.295404,...,-0.263659,-0.260815,-0.259251,3.843510,-0.245681,-0.250671,-0.262627,-0.245547,-0.259513,-0.260683


In [19]:
new_model = tf.keras.models.load_model('./tensorsave/nose_model')

new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                3776      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [20]:
test_predictions = new_model.predict(all_pd)
print(test_predictions)

[[3.5649474]
 [2.0326536]
 [3.417096 ]
 [4.273077 ]
 [1.6727986]
 [3.1889753]
 [3.6472352]
 [3.8255873]
 [1.9129019]
 [3.4266994]
 [4.262159 ]
 [3.4449253]
 [2.5201483]
 [3.9550645]
 [1.8651468]
 [3.8336174]]


In [21]:
prediction_pd = pd.DataFrame(test_predictions, columns=['dange scale'])

now = datetime.datetime.now()
day = now.strftime('%Y%m%d')
hour = now.strftime('%H')

save = pd.concat([city_list, prediction_pd], axis=1)
save['date'] = day
save['hour'] = hour

SAVE_POINT = './savedata/prediction.csv'
save.to_csv(SAVE_POINT, sep=',', na_rep='NaN')